Version 1.1.0

# The task

In this assignment you will need to implement features, based on nearest neighbours. 

KNN classifier (regressor) is a very powerful model, when the features are homogeneous and it is a very common practice to use KNN as first level model. In this homework we will extend KNN model and compute more features, based on nearest neighbors and their distances. 

You will need to implement a number of features, that were one of the key features, that leaded the instructors to prizes in [Otto](https://www.kaggle.com/c/otto-group-product-classification-challenge) and [Springleaf](https://www.kaggle.com/c/springleaf-marketing-response) competitions. Of course, the list of features you will need to implement can be extended, in fact in competitions the list was at least 3 times larger. So when solving a real competition do not hesitate to make up your own features.   

You can optionally implement multicore feature computation. Nearest neighbours are hard to compute so it is preferable to have a parallel version of the algorithm. In fact, it is really a cool skill to know how to use `multiprocessing`, `joblib` and etc. In this homework you will have a chance to see the benefits of parallel algorithm. 

# Check your versions

Some functions we use here are not present in old versions of the libraries, so make sure you have up-to-date software. 

In [8]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
from multiprocessing import Pool

for p in [np, pd, sklearn, scipy]:
    print (p.__name__, p.__version__)

numpy 1.13.1
pandas 0.20.3
sklearn 0.19.0
scipy 0.19.1


The versions should be not less than:

    numpy 1.13.1
    pandas 0.20.3
    sklearn 0.19.0
    scipy 0.19.1
   
**IMPORTANT!** The results with `scipy=1.0.0` will be different! Make sure you use _exactly_ version `0.19.1`.

# Load data

Learn features and labels. These features are actually OOF predictions of linear models.

In [9]:
train_path = '../readonly/KNN_features_data/X.npz'
train_labels = '../readonly/KNN_features_data/Y.npy'

test_path = '../readonly/KNN_features_data/X_test.npz'
test_labels = '../readonly/KNN_features_data/Y_test.npy'

# Train data
X = scipy.sparse.load_npz(train_path)
Y = np.load(train_labels)

# Test data
X_test = scipy.sparse.load_npz(test_path)
Y_test = np.load(test_labels)

# Out-of-fold features we loaded above were generated with n_splits=4 and skf seed 123
# So it is better to use seed 123 for generating KNN features as well 
skf_seed = 123
n_splits = 4

<h2>How to use KNN for feature generation</h2>

In [64]:
X[1:2]

<1x52418 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

Below you need to implement features, based on nearest neighbors.

In [65]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=10, 
                                      n_jobs=1, 
                                      algorithm='brute')

here n_neighbors is the max number of neighbors to compute

In [66]:
XX = X[:100]

In [67]:
res = nn.fit(XX)

predict neighbor for only one X point (XX[1:2])

In [68]:
neighs_dist, neighs =res.kneighbors(XX[1:2])

In [69]:
print(neighs, neighs_dist)

[[ 1 95 29 87 59 72 41 43 18 94]] [[ 0.          1.41421356  1.41421356  1.41421356  1.41421356  1.41421356
   1.41421356  1.41421356  1.41421356  1.41421356]]


neighs is the index of the 10 closests neighbor to XX[1:2]<br>
neighs_dist is the distances of the 10 closests neighbor to XX[1:2]

In [59]:
YY = Y[:100]

In [126]:
neighs_y = YY[neighs[0]]

In [127]:
print(neighs_y)

[26  0  0 19 22 14  8 11  1 13]


this are the labels for the 10 closest neighbors

In [128]:
k = 5
k_closest_neighs_y = neighs_y[1:k + 1]
print(k_closest_neighs_y)

[ 0  0 19 22 14]


this are the labels of the k closest neighbors to XX[1:2]

In [125]:
classes_count = np.bincount(k_closest_neighs_y)
print(classes_count)

[2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1]


for every class (where index of classes_count is the class number) the number of times it appear in k_closest_neight_y

In [79]:
n_classes = 30

In [99]:
feats = [0 for _ in range(n_classes)]
class_counts = np.bincount(neighs_y[1:k+1])
total_classes = sum(class_counts)
for class_number in range(n_classes):
    if class_number < len(class_counts):
        feats[class_number] = class_counts[class_number] / total_classes
assert len(feats) == n_classes

 1. Fraction of objects of every class.
               It is basically a KNNСlassifiers predictions.

               Take a look at `np.bincount` function, it can be very helpful
               Note that the values should sum up to one

In [100]:
print(feats)

[0.40000000000000002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.20000000000000001, 0.0, 0.0, 0.0, 0.0, 0.20000000000000001, 0.0, 0.0, 0.20000000000000001, 0, 0, 0, 0, 0, 0, 0]


In [101]:
assert sum(feats) == 1

In [112]:
neighs_y == neighs_y[0]

array([ True, False, False, False, False, False, False, False, False, False], dtype=bool)

In [118]:
x = np.arange(9.).reshape(3, 3)
print(x)
np.where(x > 5)

[[ 0.  1.  2.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]]


(array([2, 2, 2]), array([0, 1, 2]))

for point 2. I dont'really know how to use np.where ...

In [124]:
neighs_y = [1,1,1,5,2,7,5]
'''
    2. Same label streak: the largest number N, 
       such that N nearest neighbors have the same label.

       What can help you: `np.where`
'''

res = 1
first_class = neighs_y[0]
for i in range(1,len(neighs_y)):
    if neighs_y[i] == first_class: res += 1
    else: break
print(res)

3


In [136]:
neighs_y = YY[neighs[0]]
neighs_dist = neighs_dist[0]

In [137]:
print(neighs_y)
print(neighs_dist)

[26  0  0 19 22 14  8 11  1 13]
[ 0.          1.41421356  1.41421356  1.41421356  1.41421356  1.41421356
  1.41421356  1.41421356  1.41421356  1.41421356]


In [138]:
'''
    3. Minimum distance to objects of each class
       Find the first instance of a class and take its distance as features.

       If there are no neighboring objects of some classes, 
       Then set distance to that class to be 999.

       `np.where` might be helpful
'''
feats = [999 for _ in range(n_classes)]
for i, _class in enumerate(neighs_y):
    feats[_class] = min(neighs_dist[i], feats[_class])

print(feats)
assert len(feats) == n_classes

[1.4142135623730949, 1.4142135623730949, 999, 999, 999, 999, 999, 999, 1.4142135623730949, 999, 999, 1.4142135623730949, 999, 1.4142135623730949, 1.4142135623730949, 999, 999, 999, 999, 1.4142135623730949, 999, 999, 1.4142135623730949, 999, 999, 999, 0.0, 999, 999, 999]


In [142]:
min(neighs_dist[1:])

1.4142135623730949

In [144]:
'''
    4. Minimum *normalized* distance to objects of each class
       As 3. but we normalize (divide) the distances
       by the distance to the closest neighbor.

       If there are no neighboring objects of some classes, 
       Then set distance to that class to be 999.

       Do not forget to add self.eps to denominator.
'''
distance_to_closest = min(neighs_dist[1:]) + 1e-6
feats = [999 / (distance_to_closest) for _ in range(n_classes)] # should I normalize also 999?
for i, _class in enumerate(neighs_y):
    feats[_class] = min(neighs_dist[i] / distance_to_closest, feats[_class])

print(feats)
assert len(feats) == n_classes

[0.99999929289371892, 0.99999929289371892, 706.39917490571429, 706.39917490571429, 706.39917490571429, 706.39917490571429, 706.39917490571429, 706.39917490571429, 0.99999929289371892, 706.39917490571429, 706.39917490571429, 0.99999929289371892, 706.39917490571429, 0.99999929289371892, 0.99999929289371892, 706.39917490571429, 706.39917490571429, 706.39917490571429, 706.39917490571429, 0.99999929289371892, 706.39917490571429, 706.39917490571429, 0.99999929289371892, 706.39917490571429, 706.39917490571429, 706.39917490571429, 0.0, 706.39917490571429, 706.39917490571429, 706.39917490571429]


In [149]:
'''
    5. 
       5.1 Distance to Kth neighbor
           Think of this as of quantiles of a distribution
       5.2 Distance to Kth neighbor normalized by 
           distance to the first neighbor

       feat_51, feat_52 are answers to 5.1. and 5.2.
       should be scalars

       Do not forget to add self.eps to denominator.
'''
for k in [2,4,8]:

    feat_51 = neighs_dist[k - 1] #distances is zero based
    feat_52 = neighs_dist[k - 1] / distance_to_closest

print([[feat_51, feat_52]])

[[1.4142135623730949, 1.0]]


In [153]:
k = 10
neighs_dist[k - 1]

1.4142135623730949

In [154]:
len(neighs_dist)

10

In [155]:
classes_count

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1])

In [ ]:
'''
    6. Mean distance to neighbors of each class for each K from `k_list` 
           For each class select the neighbors of that class among K nearest neighbors 
           and compute the average distance to those objects

           If there are no objects of a certain class among K neighbors, set mean distance to 999

       You can use `np.bincount` with appropriate weights
       Don't forget, that if you divide by something, 
       You need to add `self.eps` to denominator.
'''
for k in self.k_list:

    # YOUR CODE GOES IN HERE

    assert len(feats) == self.n_classes

------------

In [18]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors
from multiprocessing import Pool

import numpy as np


class NearestNeighborsFeats(BaseEstimator, ClassifierMixin):
    '''
        This class should implement KNN features extraction 
    '''
    def __init__(self, n_jobs, k_list, metric, n_classes=None, n_neighbors=None, eps=1e-6):
        self.n_jobs = n_jobs
        self.k_list = k_list
        self.metric = metric
        
        if n_neighbors is None:
            self.n_neighbors = max(k_list) 
        else:
            self.n_neighbors = n_neighbors
            
        self.eps = eps        
        self.n_classes_ = n_classes
    
    def fit(self, X, y):
        '''
            Set's up the train set and self.NN object
        '''
        # Create a NearestNeighbors (NN) object. We will use it in `predict` function 
        self.NN = NearestNeighbors(n_neighbors=max(self.k_list), 
                                      metric=self.metric, 
                                      n_jobs=1, 
                                      algorithm='brute' if self.metric=='cosine' else 'auto')
        self.NN.fit(X)
        
        # Store labels 
        self.y_train = y
        
        # Save how many classes we have
        self.n_classes = np.unique(y).shape[0] if self.n_classes_ is None else self.n_classes_
        
        
    def predict(self, X):       
        '''
            Produces KNN features for every object of a dataset X
        '''
                
        def compute_feats(_X):
            test_feats = []
            for i in range(_X.shape[0]):
                test_feats.append(self.get_features_for_one(_X[i:i+1]))
            return test_feats
                
        if self.n_jobs == 1:
            test_feats = compute_feats(X)
        else:
            pool = Pool(self.n_jobs)
            splitted = []
            prev = 0
            for x in xrange(self.n_jobs + 1):
                i = prev + (X.shape[0]/self.n_jobs)
                splitted.append(X[prev:i])
                prev = i
            res = pool.map(compute_feats, splitted)
            test_feats = [elem for sublist in res for elem in sublist]
            
        return np.vstack(test_feats)
        
        
    def get_features_for_one(self, x):
        '''
            Computes KNN features for a single object `x`
        '''

        NN_output = self.NN.kneighbors(x)
        
        # Vector of size `n_neighbors`
        # Stores indices of the neighbors
        neighs = NN_output[1][0]
        
        # Vector of size `n_neighbors`
        # Stores distances to corresponding neighbors
        neighs_dist = NN_output[0][0] 

        # Vector of size `n_neighbors`
        # Stores labels of corresponding neighbors
        neighs_y = self.y_train[neighs] 
        
        ## ========================================== ##
        ##              YOUR CODE BELOW
        ## ========================================== ##
        
        # We will accumulate the computed features here
        # Eventually it will be a list of lists or np.arrays
        # and we will use np.hstack to concatenate those
        return_list = [] 
        
        
        ''' 
            1. Fraction of objects of every class.
               It is basically a KNNСlassifiers predictions.

               Take a look at `np.bincount` function, it can be very helpful
               Note that the values should sum up to one
        '''
        for k in self.k_list:
            feats = [0 for _ in range(self.n_classes)]
            class_counts = np.bincount(neighs_y[1:k+1]) # distance is zero based, might get out of bound
            total_classes = sum(class_counts)
            for class_number in range(self.n_classes):
                if class_number < len(class_counts):
                    feats[class_number] = class_counts[class_number] / total_classes
            
            assert len(feats) == self.n_classes
            assert sum(feats) == 1
            return_list += [feats]
        
        
        '''
            2. Same label streak: the largest number N, 
               such that N nearest neighbors have the same label.
               
               What can help you: `np.where`
        '''
        res = 1
        first_class = neighs_y[0]
        for i in range(1,len(neighs_y)):
            if neighs_y[i] == first_class: res += 1
            else: break
        feats = [res]
        
        assert len(feats) == 1
        return_list += [feats]
        
        '''
            3. Minimum distance to objects of each class
               Find the first instance of a class and take its distance as features.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.

               `np.where` might be helpful
        '''
        feats = [999 for _ in range(self.n_classes)]
        for i, _class in enumerate(neighs_y):
            feats[_class] = min(neighs_dist[i], feats[_class])

        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            4. Minimum *normalized* distance to objects of each class
               As 3. but we normalize (divide) the distances
               by the distance to the closest neighbor.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.
               
               Do not forget to add self.eps to denominator.
        '''
        distance_to_closest = min(neighs_dist[1:])
        default_val = 999 / (distance_to_closest + self.eps) # should I normalize also 999?
        feats = [default_val for _ in range(self.n_classes)] 
        for i, _class in enumerate(neighs_y):
            feats[_class] = min(neighs_dist[i] / (distance_to_closest + self.eps), feats[_class])

        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            5. 
               5.1 Distance to Kth neighbor
                   Think of this as of quantiles of a distribution
               5.2 Distance to Kth neighbor normalized by 
                   distance to the first neighbor
               
               feat_51, feat_52 are answers to 5.1. and 5.2.
               should be scalars
               
               Do not forget to add self.eps to denominator.
        '''
        for k in self.k_list:
            
            feat_51 = neighs_dist[k - 1] #distances is zero based
            feat_52 = neighs_dist[k - 1] / distance_to_closest

            return_list += [[feat_51, feat_52]]
        
        '''
            6. Mean distance to neighbors of each class for each K from `k_list` 
                   For each class select the neighbors of that class among K nearest neighbors 
                   and compute the average distance to those objects
                   
                   If there are no objects of a certain class among K neighbors, set mean distance to 999
                   
               You can use `np.bincount` with appropriate weights
               Don't forget, that if you divide by something, 
               You need to add `self.eps` to denominator.
        '''
        for k in self.k_list:
            
            # YOUR CODE GOES IN HERE
            
            assert len(feats) == self.n_classes
            return_list += [feats]
        
        
        # merge
        knn_feats = np.hstack(return_list)
        
        assert knn_feats.shape == (239,) or knn_feats.shape == (239, 1)
        return knn_feats

SyntaxError: invalid syntax (<ipython-input-18-aae5a21ce71e>, line 120)

## Sanity check

To make sure you've implemented everything correctly we provide you the correct features for the first 50 objects.

In [ ]:
# a list of K in KNN, starts with one 
k_list = [3, 8, 32]

# Load correct features
true_knn_feats_first50 = np.load('../readonly/KNN_features_data/knn_feats_test_first50.npy')

# Create instance of our KNN feature extractor
NNF = NearestNeighborsFeats(n_jobs=1, k_list=k_list, metric='minkowski')

# Fit on train set
NNF.fit(X, Y)

# Get features for test
test_knn_feats = NNF.predict(X_test[:50])

# This should be zero
print ('Deviation from ground thruth features: %f' % np.abs(test_knn_feats - true_knn_feats_first50).sum())

deviation =np.abs(test_knn_feats - true_knn_feats_first50).sum(0)
for m in np.where(deviation > 1e-3)[0]: 
    p = np.where(np.array([87, 88, 117, 146, 152, 239]) > m)[0][0]
    print ('There is a problem in feature %d, which is a part of section %d.' % (m, p + 1))

Now implement parallel computations and compute features for the train and test sets. 

## Get features for test

Now compute features for the whole test set.

In [ ]:
for metric in ['minkowski', 'cosine']:
    print (metric)
    
    # Create instance of our KNN feature extractor
    NNF = NearestNeighborsFeats(n_jobs=4, k_list=k_list, metric=metric)
    
    # Fit on train set
    NNF.fit(X, Y)

    # Get features for test
    test_knn_feats = NNF.predict(X_test)
    
    # Dump the features to disk
    np.save('data/knn_feats_%s_test.npy' % metric , test_knn_feats)

## Get features for train

Compute features for train, using out-of-fold strategy.

In [ ]:
# Differently from other homework we will not implement OOF predictions ourselves
# but use sklearn's `cross_val_predict`
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

# We will use two metrics for KNN
for metric in ['minkowski', 'cosine']:
    print (metric)
    
    # Set up splitting scheme, use StratifiedKFold
    # use skf_seed and n_splits defined above with shuffle=True
    skf = # YOUR CODE GOES HERE
    
    # Create instance of our KNN feature extractor
    # n_jobs can be larger than the number of cores
    NNF = NearestNeighborsFeats(n_jobs=4, k_list=k_list, metric=metric)
    
    # Get KNN features using OOF use cross_val_predict with right parameters
    preds = # YOUR CODE GOES HERE
    
    # Save the features
    np.save('data/knn_feats_%s_train.npy' % metric, preds)

# Submit

If you made the above cells work, just run the following cell to produce a number to submit.

In [ ]:
s = 0
for metric in ['minkowski', 'cosine']:
    knn_feats_train = np.load('data/knn_feats_%s_train.npy' % metric)
    knn_feats_test = np.load('data/knn_feats_%s_test.npy' % metric)
    
    s += knn_feats_train.mean() + knn_feats_test.mean()
    
answer = np.floor(s)
print (answer)

Submit!

In [ ]:
from grader import Grader

grader.submit_tag('statistic', answer)

STUDENT_EMAIL = # EMAIL HERE
STUDENT_TOKEN = # TOKEN HERE
grader.status()

grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)